# Surrogate-Driven Poisoning Attack Walkthrough


This notebook demonstrates how to launch and inspect the surrogate-driven poisoning attack
introduced in **Module 4**. It mirrors the CLI scripts so you can tune hyperparameters and
visualise the impact of adversarial clients directly in a notebook session.



## 1. Environment setup

The next cell infers the project root and ensures the `4_Adversarial_FL` package is importable.
Adjust the path if you copied this notebook elsewhere.


In [ ]:
from pathlib import Path
import sys

REPO_ROOT = (Path("..").resolve().parent)
if not (REPO_ROOT / "4_Adversarial_FL").exists():
    raise RuntimeError(f"Couldn't locate 4_Adversarial_FL from {REPO_ROOT}")
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
REPO_ROOT


## 2. Load the baseline configuration

The baseline YAML mirrors the template checked into `configs/`. Feel free to modify the
returned dictionary before launching the experiment.


In [ ]:
from pathlib import Path
from pprint import pprint

from 4_Adversarial_FL.black_box_runner import load_config

CONFIG_PATH = REPO_ROOT / "4_Adversarial_FL" / "configs" / "surrogate_attack.yaml"
config = load_config(CONFIG_PATH)
pprint(config)


## 3. (Optional) Run a small-scale experiment

Set `RUN_EXPERIMENT = True` to trigger a lightweight training job inside the notebook. This
assumes the dataset specified in the config is already downloaded. The run can take several
minutes depending on the model backbone and dataset.


In [ ]:
import json

from 4_Adversarial_FL.black_box_runner import run_from_config

RUN_EXPERIMENT = False
ARTIFACTS_DIR = REPO_ROOT / "artifacts"
ARTIFACTS_DIR.mkdir(exist_ok=True)
RESULTS_PATH = ARTIFACTS_DIR / "surrogate_metrics.json"

if RUN_EXPERIMENT:
    runner = run_from_config(config)
    results = runner.results
    RESULTS_PATH.write_text(json.dumps(results, indent=2))
else:
    if RESULTS_PATH.exists():
        results = json.loads(RESULTS_PATH.read_text())
    else:
        results = {"accuracy": [], "loss": []}

results


## 4. Visualise accuracy and loss

If you skipped execution, populate `artifacts/surrogate_metrics.json` by running the CLI script
first: `python -m 4_Adversarial_FL.scripts.run_surrogate_attack --results artifacts/surrogate_metrics.json`.


In [ ]:
import matplotlib.pyplot as plt

rounds = list(range(1, len(results.get("accuracy", [])) + 1))
if rounds:
    plt.figure(figsize=(8, 4))
    plt.plot(rounds, results.get("accuracy", []), label="Accuracy")
    plt.plot(rounds, results.get("loss", []), label="Loss")
    plt.xlabel("Communication Round")
    plt.ylabel("Metric")
    plt.title("Surrogate Attack Metrics")
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No results available yet. Run an experiment or load metrics from disk.")


## 5. Next steps

- Try alternative attack strengths by editing `config['malicious']['attack']`.
- Switch the model backbone (e.g. MobileNetV2) and compare convergence.
- Increase `num_rounds` once you're confident the pipeline works end-to-end.
